In [1]:
import numpy as np
import pandas as pd

from pynba import possessions_from_file
from pynba.rapm import TimeDecayedRAPM, _RAPMData

In [2]:
half_life = 1.4
off_prior = 2.75
def_prior = 2.2

league_years = {
    "wnba": range(2009, 2023),
    "nba": range(2001, 2024),
}
season_type = "Regular Season"

for league, years in league_years.items():
    possessions = pd.concat([possessions_from_file(league, year, season_type) for year in years])
    rapm_data = _RAPMData.from_possessions(possessions)
    results = []
    dates = np.sort(np.unique(rapm_data.time))
    for date in dates:
        sub_data = rapm_data[rapm_data.time <= date]
        time_decayed_rapm = TimeDecayedRAPM.from_data(sub_data, date, half_life, off_prior, def_prior)
        stats = time_decayed_rapm.stats.copy()
        stats["date"] = date
        stats = stats.loc[stats["poss"] > 0, :]
        results.append(stats)
    
    df = pd.concat(results)
    df.to_csv(f"historical_{league}_rapm.csv")